Global robustness of lenet5 model (mnist dataset)

In [2]:
from keras.datasets import mnist 
import matplotlib.pyplot as plt 
from keras.optimizers import Adam 
from keras import utils 
import numpy as np 
import math
from tensorflow.keras.models import load_model 

In [3]:
dataset = mnist
(x_train, y_train), (x_test, y_test) = dataset.load_data() #translate the data to image 

x_train = x_train.reshape(x_train.shape[0], -1).astype("float64") 
x_test = x_test.reshape(x_test.shape[0], -1).astype("float64")

#normalization 
x_train /= 255 
x_test /= 255

#translate y_train and y_test to “one hot” form 
y_train = utils.to_categorical(y_train) 

x_train = x_train.reshape(-1, 28, 28, 1) 
x_test = x_test.reshape(-1, 28, 28, 1)
lenet5 = load_model('Model/lenet5_mnist.h5')#load lenet5 model trained with mnist dataset 
lenet5.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy']) 


In [16]:
beta = 0.11200
sample(x_test,y_test,beta)

Beta is reduced by 0.0001, the new beta is 0.1119
Beta is reduced by 0.0001, the new beta is 0.1118
Beta is reduced by 0.0001, the new beta is 0.1117
Beta is reduced by 0.0001, the new beta is 0.11159999999999999
Beta is reduced by 0.0001, the new beta is 0.11149999999999999
Beta is reduced by 0.0001, the new beta is 0.11139999999999999
Beta is reduced by 0.0001, the new beta is 0.11129999999999998
Beta is reduced by 0.0001, the new beta is 0.11119999999999998
Beta is reduced by 0.0001, the new beta is 0.11109999999999998
Beta is reduced by 0.0001, the new beta is 0.11099999999999997
Beta is reduced by 0.0001, the new beta is 0.11089999999999997
Beta is reduced by 0.0001, the new beta is 0.11079999999999997
Beta is reduced by 0.0001, the new beta is 0.11069999999999997
Beta is reduced by 0.0001, the new beta is 0.11059999999999996
Beta is reduced by 0.0001, the new beta is 0.11049999999999996
Beta is reduced by 0.0001, the new beta is 0.11039999999999996
Beta is reduced by 0.0001, the 

0.10629999999999984

Global robustness Sample Algorithm

In [11]:
def sample(x_test,y_test,beta):
    
    
    mean = np.zeros(2)  
    cov = np.eye(2)
    
    for i in range(x_test.shape[0]):
        count = 0
        while True:
            p = np.random.multivariate_normal(mean, cov,(28,28)) #sample formula
            p = p.reshape(-1,28,28,1)
            neighbour = p[0] + x_test[i]*255
            neighbour /= 255
            distance=np.linalg.norm(p[0]) 
            distance /= 255
            if (distance < beta or distance == beta):
                #print(distance)
                output = lenet5.predict(neighbour.reshape(-1, 28, 28, 1),batch_size=128)[0]
                proba = softmax(output)
                pred = np.argmax(proba)
                label = y_test[i]
                if(pred == label): 
                    count+=1
                else:  #if there are some iamges inside beta cannot have the same label, beta will be decreased
                    beta = beta-0.0001
                    print("Beta is reduced by 0.0001, the new beta is",beta)  
                    i-=1
                    break
                    
            if(count<600):  
                continue
                
            if(count==600):  
                #local robustness formula satisfied
                break
                
    print("global robustness formula satisfied") 
    print("max safe distance:",beta)

def softmax(x):
    e_x = np.exp(x-np.max(x))
    return e_x / e_x.sum()